# Exploring MNE's data structure
---

## Learning Objectives
- Import raw EEG data using MNE
- Examine the structure of an MNE `Raw` object
- Access and view different attributes of a `Raw` data object
- Visualize the positions of EEG electrodes 


---
## Introduction

In this lesson, we will load a raw EEG data file using MNE and gain an understanding of how MNE stores data. This is an important first step to working with data in MNE.


## Load pacakges

In [64]:
import mne
import numpy as np
import matplotlib.pyplot as plt

## Import raw data

For EEG, raw data is typically stored in a single file containing continuous EEG data. There are many different file formats for EEG data; most of these are developed by the manufacturer or a particular EEG system. Fortunately, MNE provides functions to import data from most common EEG systems. 

In the present case, we are working with data from a system sold by Brain Products, whose software is called *Brain Vision*. So we will use MNE's `read_raw_brainvision()` function. As described previously, many of MNE's functions are nested inside subfolders (modules); for example, all of the file input-output (I/O) routines are in the module `mne.io`. So when we call the `read_raw_brainvision()` function, we have to do so as `mne.io.read_raw_brainvision()`. The one required argument for this function is the file name, and that's all we need for now: 

In [65]:
raw = mne.io.read_raw_brainvision('data/P4.vhdr')

Extracting parameters from data/P4.vhdr...
Setting channel info structure...


/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39870/1158076128.py:1: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision('data/P4.vhdr')


You may notice above, and throughout this lesson, that MNE provides a lot output, and often some concerning-looking Python warning messages. Most of these warnings are safely ignored. In subsequent lessons we'll see how to reduce the amount of output MNE generates, since it often consumes a lot of space in a notebook, and is not terribly useful.

### View raw data attributes

We now have our raw EEG data represented in Python as an MNE `Raw` object. We can view it's basic information by asking for the `.info` property:

In [66]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, F3, F7, C3, P3, TP9, Fz, Cz, Fp2, F4, F8, C4, P4, TP10, Pz, Oz
 chs: 16 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2017-03-08 09:05:29 UTC
 nchan: 16
 projs: []
 sfreq: 500.0 Hz
>

Although some of the information is not available, this shows us a few important things about our data set, including that there are 16 EEG channels (each channel contains the data from one EEG electrode), that the sampling rate is 500 Hz (i.e., EEG data were sampled 500 times per second, so we have a data point every 2 ms), and that the data were filtered during data collection between 0.01–80 Hz (more on filtering below).

The information provided above is actually a subset of the information stored in the `Raw` object's `info`. We can use the `.keys()` method to see all of the fields available in `info`:

In [67]:
raw.info.keys()

dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])

We can access the values stored with any of these info attributes by putting it in square brackets, like this command to access the names of all of the channels (electrodes) in the data set:

In [68]:
raw.info['ch_names']

['Fp1',
 'F3',
 'F7',
 'C3',
 'P3',
 'TP9',
 'Fz',
 'Cz',
 'Fp2',
 'F4',
 'F8',
 'C4',
 'P4',
 'TP10',
 'Pz',
 'Oz']

The `info` is a snapshot of a few pieces of information that researchers may want to know about a data file, but it's far less than all the information stored in the MNE `Raw` object. To see the entire contents of the object, we can ask for it's `__dict__` (note that there are two underscores below `dict` and two after). Recall that `Raw` is a Python class, and any instance of the `Raw` class (such as our `raw` data here) is a Python object. Any Python object has an atrribute `__dict__`, and this attribute contains a dictionary of all of the object's attributes, with keys being the attribute names, and values being the information stored for that attribute.

In [69]:
raw.__dict__

{'preload': False,
 '_last_samps': array([682589]),
 '_first_samps': array([0]),
 'info': <Info | 7 non-empty values
  bads: []
  ch_names: Fp1, F3, F7, C3, P3, TP9, Fz, Cz, Fp2, F4, F8, C4, P4, TP10, Pz, Oz
  chs: 16 EEG
  custom_ref_applied: False
  highpass: 0.0 Hz
  lowpass: 80.0 Hz
  meas_date: 2017-03-08 09:05:29 UTC
  nchan: 16
  projs: []
  sfreq: 500.0 Hz
 >,
 'buffer_size_sec': 1.0,
 'verbose': None,
 '_cals': array([4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08,
        4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08,
        4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08,
        4.88281e-08]),
 '_raw_extras': [{'offsets': None,
   'fmt': 'single',
   'order': 'F',
   'n_samples': 682590,
   'orig_nchan': 16}],
 '_read_picks': [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])],
 '_read_comp_grade': None,
 '_comp': None,
 '_filenames': ['/Users/aaron/NESC3505/NESC_3505_textbook/eeg/data/P4.eeg'],
 'ori

## Where's the data?

For all the information that is in the output above, one thing you may have noticed is that there doesn't seem to be any actual EEG data! This is because, by default, MNE's routines to read raw data read the data file's *header* into memory, but not the data itself. The reason for this is that data files are often quite large, and so reading them in sometimes takes a noticeable amount of time, and consume significant memory. The header of a raw EEG data file contains its metadata (information about the file), as you saw above in printing out the `__dict__` this is often enough for MNE to perform some operations on the data. At some point, if you want to perform operations on the EEG data, then MNE will automatically load in the data to work on it. 

We can force MNE to load in the data when we read the file, with the `preload` kwarg. We'll do this now so that we can look at the data:

In [70]:
raw = mne.io.read_raw_brainvision(raw_file, preload=True)

raw.__dict__

Extracting parameters from data/P4.vhdr...
Setting channel info structure...
Reading 0 ... 682589  =      0.000 ...  1365.178 secs...


/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39870/4236418496.py:1: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(raw_file, preload=True)


{'_last_samps': array([682589]),
 '_first_samps': array([0]),
 'info': <Info | 7 non-empty values
  bads: []
  ch_names: Fp1, F3, F7, C3, P3, TP9, Fz, Cz, Fp2, F4, F8, C4, P4, TP10, Pz, Oz
  chs: 16 EEG
  custom_ref_applied: False
  highpass: 0.0 Hz
  lowpass: 80.0 Hz
  meas_date: 2017-03-08 09:05:29 UTC
  nchan: 16
  projs: []
  sfreq: 500.0 Hz
 >,
 'buffer_size_sec': 1.0,
 'verbose': None,
 '_cals': array([4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08,
        4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08,
        4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08, 4.88281e-08,
        4.88281e-08]),
 '_raw_extras': [{'offsets': None,
   'fmt': 'single',
   'order': 'F',
   'n_samples': 682590,
   'orig_nchan': 16}],
 '_read_picks': [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])],
 '_read_comp_grade': None,
 '_comp': None,
 '_filenames': ['/Users/aaron/NESC3505/NESC_3505_textbook/eeg/data/P4.eeg'],
 'orig_format': 'single'

Now we can see that there is a `_data` attribute, containing a NumPy array. We can access the data like so:

In [71]:
print(type(raw._data))
print(raw._data.shape)

<class 'numpy.ndarray'>
(16, 682590)


MNE represents the data as a 2D array with a row for each channel, and columns for time points. We can derive the length of the EEG recording, in seconds, by dividing the number of time points by the sampling rate:

In [72]:
scan_durn = raw._data.shape[1] / raw.info['sfreq']
print('Duration of EEG recording = ', scan_durn, 's, or', scan_durn / 60, 'min.')

Duration of EEG recording =  1365.18 s, or 22.753 min.


## Accessing subsets of data

Since the data is stored as a NumPy array, we can access specific data points using standard indexing. For example, if we wanted to select all of the data from only channel 15, we would use:

In [73]:
print(raw._data[15, :])

[-3.49426091e-05 -3.36795634e-05 -2.86693426e-05 ... -3.19992683e-01
 -3.19929304e-01 -3.19865950e-01]


Likewise, if we wanted to access only the first 10 time points of channel 8, we would use:

In [74]:
print(raw._data[8, :11])

[0.00021124 0.00021153 0.00021526 0.00022065 0.00022381 0.0002228
 0.00022086 0.00022152 0.00022406 0.00022598 0.00022798]


One challenge in working with the raw data directly in this way, is that we normally think of, and work with, EEG data in terms of units of time, such as seconds or milliseconds. However, EEG data is not always (or even usually) sampled at 1000 Hz, so each time point doesn't correspond to 1 ms. For example, in our current data set, the sampling rate was 500 Hz, so each time point reflects a period of 2 ms. This makes accessing specific time points or time ranges in the data tricky, because if you want, say, the data from 1500-1800 ms in the `raw` data, you would need to first convert those times to seconds (since `sfreq` is in samples per second), oncvert the result to `int` (since division is likely to yield a `float` result), then multiply each of those times by the sampling rate to get the correct indices. While this is not necessarily difficult, it's a bit tricky and prone to error, as well as creating complex code:

In [75]:
chan = 8 # set channel we want
start_time = int((1500 / 1000) * raw.info['sfreq'])
end_time   = int((1800 / 1000) * raw.info['sfreq'])
raw._data[chan, start_time:end_time]

array([0.00018245, 0.00018437, 0.00018864, 0.00019457, 0.00019848,
       0.00019798, 0.00019517, 0.00019331, 0.00019327, 0.00019501,
       0.00019897, 0.00020431, 0.00020803, 0.00020714, 0.0002027 ,
       0.00019861, 0.00019641, 0.00019518, 0.00019454, 0.00019453,
       0.00019386, 0.00019153, 0.00018945, 0.00018907, 0.0001888 ,
       0.00018899, 0.00019214, 0.00019711, 0.00020056, 0.00020286,
       0.00020605, 0.00020858, 0.00020825, 0.00020669, 0.00020621,
       0.00020703, 0.00020843, 0.00020945, 0.00021   , 0.00021019,
       0.0002085 , 0.00020438, 0.00020072, 0.00020058, 0.00020341,
       0.00020607, 0.00020706, 0.00020666, 0.00020411, 0.00019923,
       0.00019499, 0.00019539, 0.00020034, 0.00020524, 0.0002075 ,
       0.00020769, 0.00020545, 0.00020112, 0.00019691, 0.00019422,
       0.00019391, 0.00019645, 0.00020019, 0.00020246, 0.00020222,
       0.0001997 , 0.00019523, 0.00019023, 0.00018778, 0.00019002,
       0.00019562, 0.00020115, 0.00020264, 0.00019867, 0.00019

## Accessing data using MNE tools

Fortunately, since MNE is made to work with EEG data, it has lots of functions and methods that make it easy to do common tasks. Besides convenience, it's generally a good idea to use these because in general they provide an API that is more intuitive to use, and also create less risk that you will corrupt the data (since if you're using `._data`, it would be easy to accidentally assign new values to the array, or otherwise modify it, and Python would not warn you that you were doing something bad).

We can access the data in an MNE object with the `.get_data()` method. 


In [76]:
raw_data = raw.get_data()
print(raw_data.shape)

(16, 682590)


Moreover, `.get_data()` provides kwargs that allow us to select channels and time ranges, using more intuitive units like channel labels, and times, rathter than NumPy array indices. `picks` is used to select the channel (or channels if you pass a list), and `tmin` and `tmax` specify the range of time that you want:

In [78]:
raw.get_data(picks=['Fz', 'Cz'], tmin=1.5, tmax=1.8)

TypeError: get_data() got an unexpected keyword argument 'tmin'

In [79]:
mne.__version__

'0.23.0'

### Set electrode positions

One thing that can be useful at this stage is visualizing the electrode positions on the head. Typically, EEG studies try to position electrode in a fairly evenly-spaced way over the top of the head. However, the locations can depend someone on what ERP [components](./components) are predicted for the study, and how many electrodes are available. 

MNE provides a `.plot_sensors()` method for visualizing sensor locations. However, by default although our raw data contains the *names* of each channel, it does not contain information regarding where each channel was located. Fortunately, in this and most EEG studies, the electrode positions were based on the International 10-20 system. This was first introduced as a way of standardizing EEG electrode placement in clinical neurology, and is so-named because electrode positions are determined by first measuring the anterior-posterior and left-right dimensions of the participant's head, and then placing electrodes at positions determined by 10 and 20% increments of those measurements (E.g., electrode Cz is placed 50% of the anterior-posterior and left-right measurements). More recently, [Oostenveld and Praamstra (2001)](https://doi.org/10.1016/S1388-2457(00)00527-7) published an extension of this called the 10-5 system, that allows for a larger number of electrodes based on 5% increments. You can read more about this system and its variants on [Oostenveld's blog](https://robertoostenveld.nl/electrode/#oostenveld2001). 

Long story short, we have to first run an MNE method to look up the locations of each electrode based on its 10-20 system name, and set the positions in the raw file. We do this with `.set_montage()`, and as an argument use MNE's built-in list of 10-5 system electrode names and locations.

<div class="alert alert-block alert-info">
    
**Note:** *Montage* is a common word used to describe the arrangement of EEG electrodes on a head, or in an EEG cap. In more general usage, *montage* means "a collection of elements".</div>


In [ ]:
raw.set_montage('standard_1005')

MNE prints out the `info` for the `Raw` object again, and if you compare this output to that from earlier, you'll see that the *Digitized points* field has changed from "Not available" to "19 points". This terminology is a bit strange - "digitized points" seems more general and nonspecific than "electrode locations", for example. The reason for this is that in some cases, researchers will actually use a device to digitize the locations of electrodes when they are on a person's head, rather than relying on standard positions. Anyway, this information is stored under the `dig` attribute, which we can view with:

In [ ]:
raw.info['dig']

Each electrode location is stored as a set of (*x*, *y*, *z*) Cartesian coordinates, i.e., positions in a 3D grid, where *x* is the left-right dimension, *y* is posterior-anterior, and *z* is inferior-posterior. The (0, 0, 0) location is inside the head, located in a plane defined by the bridge of the nose (called the **nasion**) and the left and right ear canals. As shown in the figure below, *x* goes from left (negative) to right (positive), *y* goes from posterior to anterior, and *z* goes from inferior to superior. In general, you won't need to work with this coordinate system directly, but it's useful to know how the data are represented.

![cartesian coordinate system for electrode positions](images/MRI_ax_sag_cor.jpg)

### View electrode positions

MNE provides tools for viewing channel locations in both 2D and 3D:

In [ ]:
raw.plot_sensors()

### MNE plots in Jupyter notebooks

You'll notice above that MNE actually drew two copies of the figure. This is a default behaviour of MNE for many plots, and it's annoying if you don't know how to deal with it. There are two ways to ensure MNE produces only a single copy of a figure:
- end the plot command with a semicolon
- put `plt.show()` at the bottom of the cell (or at least after all plotting commands in the cell)  

In [ ]:
raw.plot_sensors();

Below we demonstrate the `plt.show()` approach, and add a kwarg to show the names of each sensor:

In [ ]:
raw.plot_sensors(show_names=True)
plt.show()

- view the whole raw object
- view the list of channel names
- view/extract the data 
